# Исследование надёжности заёмщиков

In [ ]:
import pandas as pd

In [4]:
data = pd.read_csv('data.csv')
display(data.describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06



**Описание данных:**
 * children — количество детей в семье;
 * days_employed — общий трудовой стаж в днях;
 * dob_years — возраст клиента в годах;
 * education — уровень образования клиента;
 * education_id — идентификатор уровня образования;
 * family_status — семейное положение;
 * family_status_id — идентификатор семейного положения;
 * gender — пол клиента;
 * income_type — тип занятости;
 * debt — имел ли задолженность по возврату кредитов;
 * total_income — ежемесячный доход;
 * purpose — цель получения кредита.


## Предобработка данных

1. Определить и заполнить пропущенные значения.
2. Заменить вещественный тип данных на целочисленный.
3. Удалить дубликаты.
4. Выделить леммы в значениях столбца с целями получения кредита.
5. Категоризировать данные.

In [4]:
data.isna().sum() #Количество пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В данных присутствуют артефакты — значения, которые не отражают действительность. Например, отрицательное количество дней трудового стажа и количество детей. Минусы перед числом могли появиться в результате ошибки в процессе заполнения данных, поэтому необходимо сделать значения столбцов 'children' и 'days_employed' положительными.

In [5]:
data['children'] = data['children'].apply(abs)
data['days_employed'] = data['days_employed'].apply(abs)

Также в общей информации о данных можно заметить, что минимальное значение для столбца 'dob_years' - возраст клиента в годах - равно нулю. 

In [6]:
print(data[data['dob_years'] == 0]['dob_years'].count()) #Количество нулей, указанных в столбце 'возраст'

101


Так как возраст клиентов это количественная переменная, можно заполнить пропущенные значения медианным.

In [7]:
dob_years_median = data.loc[data['dob_years'] != 0]['dob_years'].median() #подсчет медианного значения без учета нулей
data.loc[data['dob_years'] == 0 ,'dob_years'] = dob_years_median #заполнение нулевых значений медианными
display(data.describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,66914.728907,43.495145,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,139030.880527,12.218213,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,34.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,43.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Пропуски присутствуют в значениях столбцов 'days_employed' и 'total_income'.
Причиной пропусков значений в этих столбцах мог стать тот факт, что у клиентов банка отсутствует трудовой стаж или официальный доход. 

Стаж и доход - количественные переменные. Если сделать предположение, что уровень дохода зависит от типа занятости, то пропущенные значения можно заполнить медианным значением для каждой группы по типу занятости.

In [8]:
groups = data['income_type'].unique()
median_total_income = data.groupby('income_type')['total_income'].median()
median_days_employed = data.groupby('income_type')['days_employed'].median()

for group in groups:
    data.loc[(data['income_type']==group) & (data['total_income'].isna()), 'total_income'] = median_total_income[group]
    data.loc[(data['income_type']==group) & (data['days_employed'].isna()), 'days_employed'] = median_days_employed[group]

data.isna().sum() #Проверка заполнения пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [9]:
data.dtypes

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Столбцы 'children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt', исходя из описания данных, принимают только целочисленные значения. Поэтому необходимо изменить тип данных в этих колонках с 'float64' на 'int'.

In [10]:
columns = [ 'children','days_employed','dob_years','education_id','family_status_id','debt']
for column in columns:
    data[column] = data[column].astype(int)
display(data.dtypes)

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [11]:
data.duplicated().sum() #подсчет количества строк-дубликатов

54

В данных присутствует 54 строки-дубликата. Они могли появиться в результате ошибки заполнения данных сотрудниками банка, или, например, один и тот же клиент мог обратиться в банк для получения кредита несколько раз. В течение этого периода его трудовой стаж, уровень дохода и семейное положение могли не измениться, вследствие чего одинаковые данные о нем могли быть учтены повторно.

In [12]:
data = data.drop_duplicates().reset_index(drop = True)
display(data.duplicated().sum())

0

Необходимо рассмотреть, какие цели получения кредита присутствуют в данных и как распределяется частота их упоминания.

In [13]:
data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Можно выделить несколько причин, по которым люди берут кредит, но в данных они представлены с помощью разных формулировок, что затрудняет анализ. Поэтому нужно произвести лемматизацию столбца 'purpose' с помощью метода Mystem библиотеки pymystem3. Функция 'lem' производит лемматизацию каждой строки столбца 'purpose' и возвращает название одной из 4 категорий: жилье, образование, свадьба или автомобиль. 

In [14]:
from pymystem3 import Mystem
m = Mystem()

def lem(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'жилье' in lem_purpose or 'жильё' in lem_purpose or 'недвижимость' in lem_purpose or 'квартира' in lem_purpose:
        return 'жилье'
    elif 'образование' in lem_purpose:
        return 'образование'
    elif 'свадьба' in lem_purpose:
        return 'свадьба'
    elif 'авто' in lem_purpose or 'автомобиль' in lem_purpose:
        return 'автомобиль'

data['purpose'] = data.apply(lem, axis = 1)
display(data['purpose'])

0              жилье
1         автомобиль
2              жилье
3        образование
4            свадьба
            ...     
21466          жилье
21467     автомобиль
21468          жилье
21469     автомобиль
21470     автомобиль
Name: purpose, Length: 21471, dtype: object

В таблице присутствуют две пары столбцов, которые отображают одинаковые данные и могут быть вынесены в отдельные таблицы: 'family_status_id' и 'family_status', 'education_id' и 'education'. Создание дополнительных таблиц производится с помощью извлечения необходимых столбцов из главной таблицы и использования методов drop_duplicates и reset_index.

In [15]:
family_status_table = data[['family_status_id','family_status']].drop_duplicates().reset_index(drop = True)
print(family_status_table) 

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


In [16]:
education_table = data.loc[:,['education_id','education']]
education_table['education'] = education_table['education'].str.lower()
education_table  = education_table.drop_duplicates().reset_index(drop = True)
print(education_table) 

   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень


In [17]:
data = data.drop(columns = ['education','family_status'])
display(data.columns)

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

Также для дальнейшего анализа необходимо разделить клиентов на группы по уровню дохода. Для этого нужно оценить диапазон значений столбца 'total_income'.

In [18]:
print(data['total_income'].mean())
print(data['total_income'].median())
print(data['total_income'].max())

165295.8877638656
142594.39684740017
2265604.028722744


Можно выделить группы: близкий к медианному доходу, ниже медианного и выше медианного. Далее производится категоризация c помощью функции income_level.

In [19]:
median_income_low = data['total_income'].median() - (data['total_income'].median())/2
median_income_high = data['total_income'].median() + (data['total_income'].median())/2

def income_level(row):
    if row['total_income'] < median_income_low:
        return 'ниже среднего'
    elif median_income_low <= row['total_income'] <= median_income_high:
        return 'средний'
    elif row['total_income'] > median_income_high:
        return 'высокий'

data['total_income_id'] = data.apply(income_level, axis = 1)


display(data['total_income_id'])

0        высокий
1        средний
2        средний
3        высокий
4        средний
          ...   
21466    высокий
21467    средний
21468    средний
21469    высокий
21470    средний
Name: total_income_id, Length: 21471, dtype: object

## Ответы на вопросы заказчика

Влияние различных факторов на возврат кредита в срок будет оцениваться по значению столбца 'debt'. Строки в этом столбце принимают значения: '0' - клиент не имел задолженности, '1' - клиент имел задолженность.

Зависимость между тем или иным фактором и возвратом кредита в срок будет определяться с помощью отношения числа должников в каждой из групп к общему числу клиентов в группе.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
#процент должников для клиентов без детей:
print(data.loc[data['children']==0]['debt'].sum()/data.loc[data['children']==0]['debt'].count())
#процент должников для клиентов с детьми:
print(data.loc[data['children']!=0]['debt'].sum()/data.loc[data['children']!=0]['debt'].count())

0.07535266179910682
0.09206952743074416


**Вывод**

Вероятность возврата кредита в срок для клиентов с детьми выше, чем для клиентов без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
data.groupby('family_status_id')['debt'].sum()/data['family_status_id'].value_counts().sort_index()
#значения в знаменателе после применения value_counts были 
#отсортированы по убыванию суммы значений 'debt', поэтому применяется sort_index

family_status_id
0    0.075421
1    0.093202
2    0.065693
3    0.071130
4    0.097509
dtype: float64

**Вывод**

Наиболее высокие проценты задолженности у клиентов с идентификатором столбца 'family_status_id' равным 4 ('Не женат / не замужем') и 1 ('гражданский брак'). Наименьший - у клиентов с  семейным положением 'вдовец / вдова'.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
data.groupby('total_income_id')['debt'].sum()/data['total_income_id'].value_counts().sort_index()

total_income_id
высокий          0.070370
ниже среднего    0.069031
средний          0.085281
dtype: float64

**Вывод**

Наибольшее значение задолженностей у категории дохода 'средний', поэтому нельзя сказать о значительной корреляции между уровнем дохода и возвратом кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
data.groupby('purpose')['debt'].sum()/data['purpose'].value_counts().sort_index()

purpose
автомобиль     0.093547
жилье          0.072314
образование    0.092177
свадьба        0.079657
dtype: float64

**Вывод**

Самые ответственные клиенты - клиенты, у которых есть дети, а также те, кто берет кредит на жилье и на свадьбу. 

Наибольшее число должников наблюдается в категории по семейному положению 'не женат/не замужем'; для категорий с целями кредита - в категориях 'автомобиль' и 'образование'.

Значительной зависимости возврата кредита в срок от уровня дохода не наблюдается.